In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import plotly.express as px
import os
from tqdm import tqdm
from torchmetrics.functional.classification import binary_stat_scores

In [ ]:
# LSTM parameters
hidden_dim = 20
n_signals = 3
N = 64

# _batch_size => m in figure 1.
train_batch_size = 64
dev_batch_size = 16
test_batch_size = 16

# Classification type (binary)
tagset_size = 1

# Set
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# Model Definition


class FaultDetector(nn.Module):
    """Information about FaultDetector"""

    def __init__(self, n_signals, hidden_dim, tagset_size):
        super(FaultDetector, self).__init__()
        self.lstm = nn.LSTM(n_signals, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, tagset_size)
        # self.norm = nn.BatchNorm1d(tagset_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        fc_layer = self.fc(lstm_out[:,-1, :])
        # norm_layer = self.norm(fc_layer)
        

        return torch.sigmoid(fc_layer)

In [ ]:
model = FaultDetector(n_signals, hidden_dim, tagset_size).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Learning rate decay (optional)
decayRate = 0.96
my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(
    optimizer=optimizer, gamma=decayRate
)

print(f"Model structure: {model}\n")

# Number of parameters
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"Number of parameters: {params}")

In [ ]:
# from utils.signalload import CSV_pandas_path
# from utils.auxfunctions import moving_window
from utils_tesis.signalload import CSV_pandas_path
from utils_tesis.auxfunctions import moving_window
import numpy as np
from itertools import repeat
import random


class Form1Dataset(torch.utils.data.Dataset):
    """Some Information about Form1Dataset"""

    def __init__(
        self,
        dataset_dir,
        signal_names,
        max_window_idx=65,
        window_length=64,
        step=1,
        test=False,
        dataset_size=999999,
    ):
        super(Form1Dataset, self).__init__()

        self.signal_names = signal_names
        self.max_window_idx = max_window_idx
        self.dataset_dir = dataset_dir
        self.test = test

        # Find all csv in folders of DataSet
        file_set = set()
        for dir_, _, files in os.walk(dataset_dir):
            for file_name in files:
                rel_dir = os.path.relpath(dir_, dataset_dir)
                rel_file = os.path.join(rel_dir, file_name)
                file_set.add(rel_file)
        csv_list = list(file_set)

        if dataset_size < len(csv_list):
            csv_list = random.sample(csv_list, dataset_size)
            # csv_list = csv_list[:dataset_size]
        self.csv_list = csv_list
        self.csv_amount = len(csv_list)
        self.windows_amount = max_window_idx * self.csv_amount
        self.window_length = window_length
        self.step = step

    def __getitem__(self, index, data_plot=False):

        # sample_settings
        window_length = self.window_length
        step = self.step

        # calculate window_idx and filename
        window_idx = index % self.max_window_idx
        csv_idx = index // self.max_window_idx
        csv_full_path = f"{self.dataset_dir}\{self.csv_list[csv_idx]}"

        # Load CSV, signal and create windows
        csv_name = os.path.basename(csv_full_path)
        signals_windows = np.empty((self.window_length, 0))

        # Load Change Event

        if "L_T" in self.csv_list[csv_idx]:
            for signal_name in self.signal_names:
                signal, t, _ = CSV_pandas_path(csv_full_path).load_data(signal_name)
                # select window
                len_ = len(signal)
                signal_window = signal[
                    -self.window_length
                    - self.max_window_idx
                    + window_idx
                    - 1 : -self.max_window_idx
                    + window_idx
                    - 1
                ]
                signal_window = np.expand_dims(signal_window, axis=1)
                signals_windows = np.append(signals_windows, signal_window, axis=1)

            t_window = moving_window(t, window_length, step)
            t_window = t_window[window_idx]
            # Create labels

            label = np.array([0])

        # Fault Event
        elif "F_T" in self.csv_list[csv_idx]:

            # For faults indices 0 and 1 don't contain fault events.
            for signal_name in self.signal_names:
                signal, t, _ = CSV_pandas_path(csv_full_path).load_data(signal_name)
                # select window
                signal_window = signal[
                    -self.window_length
                    - self.max_window_idx
                    + window_idx
                     : -self.max_window_idx
                    + window_idx
                    
                ]
                signal_window = np.expand_dims(signal_window, axis=1)
                signals_windows = np.append(signals_windows, signal_window, axis=1)

            t_window = moving_window(t, window_length, step)
            t_window = t_window[window_idx]
            # Create labels
            label = np.array([1])

        signals_windows = torch.from_numpy(np.copy(signals_windows)).float()
        label = torch.from_numpy(label).float()
        # For Dataset Visualization
        if data_plot == True:
            return signals_windows.reshape(-1).numpy(), t_window
        if self.test == True:
            return signals_windows, t_window, label, csv_name, index, window_idx

        return signals_windows, label

    def get_event(self, csv_idx):

        # Get indices
        idx_min = csv_idx * self.max_window_idx
        idx_max = ((csv_idx + 1) * self.max_window_idx) - 1

        # Get path of csv_index
        csv_full_path = f"{self.dataset_dir}\{self.csv_list[csv_idx]}"

        # Load CSV, signal and create windows
        csv_name = os.path.basename(csv_full_path)
        signals = np.zeros((0, 0))
        for idx, signal_name in enumerate(self.signal_names):
            signal, t, _ = CSV_pandas_path(csv_full_path).load_data(signal_name)
            signal = np.expand_dims(signal, axis=1)
            if idx == 0:
                signals = np.empty((len(signal), 0))
            signals = np.append(signals, signal, axis=1)
        signal = signals
        return signal, t, idx_min, idx_max, csv_name

    def len_events(self):
        return self.csv_amount

    def __len__(self):
        return self.csv_amount * self.max_window_idx


In [ ]:
def confusion_matrix_labels(pred_label, true_label):
    label = ""
    if int(pred_label) == int(true_label):
        label += "T"
    else:
        label += "F"
    if pred_label == 1:
        label += "P"
    else:
        label += "N"
    return label


confusion_matrix_pandas = np.vectorize(confusion_matrix_labels)


def confusion_matrix(
    preds: torch.FloatTensor, labels: torch.FloatTensor
) -> pd.DataFrame:
    preds = preds.detach()
    labels = labels.detach()
    data = {
        "Pred probability": torch.reshape(preds, (-1,)).cpu().numpy(),
        "Pred label": torch.reshape(torch.round(preds), (-1,)).int().cpu().numpy(),
        "True label": torch.reshape(labels, (-1,)).int().cpu().numpy(),
    }
    df = pd.DataFrame(data)
    df["Result"] = confusion_matrix_pandas(df["Pred label"], df["True label"])
    return df

def conf_matrix_metrics(conf_matrix: torch.LongTensor) -> dict:
    """
    Returns dictionary with metrics from a confusion matrix.

            Parameters:
                    conf_matrix (torch.Tensor): confusion matrix of dimension (1, 5)
                        [TP, FP, TN, FN, TP + FN]

            Returns:
                    metrics (dict): dictionary with following metrics:
                        metrics["TOTAL"] -> total amount of samples.
                        metrics["TPR"]   -> True Positive Rate,  sensibility, recall, hit-rate.
                        metrics["FPR"]   -> False Positive Rate, Fallout.
                        metrics["TNR"]   -> True Negative Rate,  specificity, selectivity
                        metrics["ACC"]   -> Accuracy.
                        metrics["PPV"]   -> Positive Predictive Value, Precision.
    """
    if conf_matrix.shape == (5,):
        conf_matrix = np.expand_dims(conf_matrix, axis=0)

    metrics = {}
    TP = int(conf_matrix[0, 0].item())
    FP = int(conf_matrix[0, 1].item())
    TN = int(conf_matrix[0, 2].item())
    FN = int(conf_matrix[0, 3].item())
    metrics["TP"] = TP
    metrics["FP"] = FP
    metrics["TN"] = TN
    metrics["FN"] = FN
    P = TP + FN
    N = TN + FP
    TOTAL = TP + FP + TN + FN
    metrics["TOTAL"] = TOTAL
    try:
        metrics["TPR"] = TP / (TP + FN)
    except ZeroDivisionError:
        metrics["TPR"] = "ZeroDivisionError"
    try:
        metrics["FPR"] = FP / (FP + TN)
        metrics["TNR"] = TN / (FP + TN)
    except ZeroDivisionError:
        metrics["FPR"] = "ZeroDivisionError"
        metrics["TNR"] = "ZeroDivisionError"
    metrics["ACC"] = (TP + TN) / (TOTAL)
    try:
        metrics["PPV"] = TP / (TP + FP)
    except ZeroDivisionError:
        print("No se puede obtener PPV, división por cero")
    return metrics

def signal_exploration(idx: int, dataset, model, plot_signal: bool = True):
    signal, t, idx_min, idx_max, csv_name = dataset.get_event(idx)
    model.eval()
    if plot_signal == True:
        plt.plot(t, signal)
        plt.show()
    conf_matrix = torch.zeros(1, 5, dtype=torch.int64).to(device)
    preds = torch.empty((0, 1)).to(device)
    labels = torch.empty((0, 1)).to(device)
    for i in range(idx_min, idx_max + 1):
        signal, y = dataset.__getitem__(i)
        y = torch.unsqueeze(y, 0).to(device)
        signal = torch.unsqueeze(signal, 0).to(device)
        pred = model(signal)
        preds = torch.cat((preds, pred), 0)
        labels = torch.cat((labels, y), 0)
        conf_matrix = conf_matrix.add(binary_stat_scores(pred, y))
    df = confusion_matrix(preds, labels)
    df.insert(loc=0, column="event_idx", value=np.repeat(idx, idx_max - idx_min + 1))
    # df.insert(loc=0, column="window_idx", value=df.index)
    # df.insert(loc=0, column="indices", value=idxs)

    return df, conf_matrix


def plot_confusion_matrix(metrics):
    z = [[metrics["TP"], metrics["FN"]], [metrics["FP"], metrics["TN"]]]
    fig = px.imshow(
        z,
        text_auto=True,
        template="seaborn",
        labels=dict(x="Predicted Label", y="Real Label", color="Predictions"),
        x=["Positive", "Negative"],
        y=["Positive", "Negative"],
        width=400,
        height=300,
    )
    fig.show()


def print_metrics(metrics):
    print(f"{'Total windows:':.<30}{metrics['TOTAL']:4}")
    print(f"{'True Positives:':.<30}{metrics['TP']:4}")
    print(f"{'False Positives:':.<30}{metrics['FP']:4}")
    print(f"{'True Negatives:':.<30}{metrics['TN']:4}")
    print(f"{'False Negatives:':.<30}{metrics['FN']:4}")
    print(f"{'Accuracy:':.<30}{metrics['ACC']*100:>6.1f}%")
    print(f"{'True Positive Rate:':.<30}{metrics['TPR']*100:>6.1f}%") 
    print(f"{'False Positive Rate:':.<30}{metrics['FPR']*100:>6.1f}%") 
    print(f"{'True Negative Rate:':.<30}{metrics['TNR']*100:>6.1f}%") 
    try:
        print(f"{'Positive Predictive Value:':.<30}{metrics['PPV']*100:>6.1f}%")
    except KeyError:
        print(f"PPV divided by 0. No positive class predicted")

In [ ]:
model.load_state_dict(torch.load('./models/R2Currents_DB2_V2.pth'))

In [ ]:


dataset_dir = "C:/Users/aherrada/OneDrive - Universidad del Norte/Uninorte/DetectionDataBase/LSTM_SEM"
current_R1A = "I: X0023A-R1A"
current_R1B = "I: X0023B-R1B"
current_R1C = "I: X0023C-R1C"
current_R2A = "I: X0004A-R2A"
current_R2B = "I: X0004B-R2B"
current_R2C = "I: X0004C-R2C"
current_R3A = "I: X0071A-R3A"
current_R3B = "I: X0071B-R3B"
current_R3C = "I: X0071C-R3C"

voltage_R1A = "V: R1A"
voltage_R1B = "V: R1B"
voltage_R1C = "V: R1C"
signal_names = [current_R2A, current_R2B, current_R2C]
max_window_idx = 62
# model.load_state_dict(torch.load('models\R3Currents_DB2.pth'))

# Create Dataset
dataset = Form1Dataset(
    dataset_dir, max_window_idx=max_window_idx, signal_names=signal_names, dataset_size=20
)

In [ ]:
conf_matrix = torch.zeros(0, 5, dtype=torch.int64).to(device)
for idx in tqdm(range(dataset.len_events())):
# for idx in tqdm(range()):
    df, CM = signal_exploration(idx, dataset, model, plot_signal=False)
    conf_matrix = torch.cat((conf_matrix, CM))
    if idx == 0:
        dataset_df = df
    else:
        dataset_df = pd.concat([dataset_df, df])

dataset_df = dataset_df.reset_index()
dataset_df = dataset_df.rename(columns={'index': 'window idx'})
conf_matrix_total = np.sum(conf_matrix.cpu().numpy(), axis=0)


# dataset_df = pd.read_parquet(f"parquet_data\R2_currents_DB2_df.parquet")
# conf_matrix_df = pd.read_parquet(f"parquet_data\R2_currents_DB2_CM_df.parquet")
# conf_matrix = np.load(f"parquet_data\R2_currents_DB2_CM.npy", allow_pickle=False)
# conf_matrix_total = np.sum(conf_matrix, axis=0)
# metrics = conf_matrix_metrics(conf_matrix_total)

In [ ]:
# df and plots settings
pd.set_option("display.float_format", "{:.2%}".format)

pd.options.plotting.backend = "matplotlib"


# print(dataset_df)
pd.set_option("display.max_rows", 2000)
false_positive = dataset_df.query('Result == "FP"')
false_negative = dataset_df.query('Result == "FN"')
# sample_df = pd.concat([sample_df, sample_df])

false_positive_plot = false_positive.groupby(["window idx"])["window idx"].count()
false_negative_plot = false_negative.groupby(["window idx"])["window idx"].count()
# print(false_positive['Pred probability'].value_counts(bins=10, sort=False))


if len(false_positive_plot) > 0:
    print("FALSE POSITVES")
    print(false_positive)
    # print(false_positive.groupby(['event_idx'])['event_idx'].count())
    false_positive.groupby(['event_idx'])['event_idx'].count().plot(kind='bar')
    plt.show()
    false_positive_plot.plot(kind="bar", edgecolor="black")
    plt.show()
    false_positive["Pred probability"].value_counts(
        bins = [i * 0.1 for i in range(5, 11)], sort=False, normalize=True
    ).plot(kind="bar")
    plt.show()
if len(false_negative_plot) > 0:
    print("FALSE NEGATIVES")
    print(false_negative)
    # print(false_negative.groupby(['event_idx'])['event_idx'].count())
    false_negative.groupby(['event_idx'])['event_idx'].count().plot(kind='bar')
    plt.show()

    false_negative_plot.plot(kind="bar", edgecolor="black")
    plt.show()
    false_negative["Pred probability"].value_counts(
        bins = [i * 0.1 for i in range(6)], sort=False, normalize=True
    ).plot(kind="bar")
    plt.show()
plt.show()

In [ ]:
# model.load_state_dict(torch.load('models\LSTMHarmonic_weights_R3Currents.pth'))
print(conf_matrix_total)
metrics = conf_matrix_metrics(conf_matrix_total)
print_metrics(metrics)
plot_confusion_matrix(metrics)

In [ ]:
event_idx = 3
print(dataset.csv_list[event_idx])
sample_df, conf_matrix = signal_exploration(event_idx, dataset, model)

In [ ]:
window_idx = 47
sample_idx = (max_window_idx * event_idx) + window_idx


def update_fig(fig):
    fig.update_traces(line_color="#EEEEEE", line_width=2)
    fig.update_layout(
        paper_bgcolor="#222831",
        plot_bgcolor="#393E46",
        font_color="whitesmoke",
    )
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="#32E0C4")
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="#32E0C4")


signal, t = dataset.__getitem__(sample_idx, data_plot=True)
signal = np.copy(signal)
signal = signal.reshape((-1, 3))
signal = signal[:, 0]
t = np.copy(t)
print(type(t))
print(type(signal))
# print(f"Window index: {sample_df.query('indices == 196').index[0]}")
fig = px.line(
    x=t,
    y=signal,
    width=600,
    height=400,
    labels=dict(x="time", y="Amplitude"),
)
# update_fig(fig)x
fig.show()
fig = px.line(
    x=sample_df.index,
    y=sample_df["Pred label"],
    width=600,
    height=400,
    labels=dict(x="Window", y="Trip Signal"),
    line_shape="hv",
)
# update_fig(fig)
fig.show()

In [ ]:
dataset_df.to_parquet('parquet_data/R2_currents_DB2_V2df.parquet')
conf_matrix_df = pd.DataFrame(conf_matrix.cpu(), columns = ['TP','FP','TF', 'FN', 'TP + FN'])
conf_matrix_df.to_parquet('parquet_data/R2_currents_DB2_CM_df_V2.parquet')
np.save('parquet_data/R2_currents_DB2_CM_V2.npy', conf_matrix.cpu())